# Unsupervised Track Clustering

## Imports

In [19]:
import itertools
import re
import string

import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.metrics import edit_distance
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import Levenshtein

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sarahamiraslani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sarahamiraslani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sarahamiraslani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Data Sourcing

### Fetch data from Wikipedia

In [4]:
def fetch_wiki_circuits(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        return response.content
    except requests.RequestException as e:
        print(f"Failed to fetch the webpage: {e}")
        return None


def parse_html_to_dataframe(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    caption = soup.find("caption", text="Formula One circuits\n")
    table = caption.find_parent("table")

    headers = [header.text.strip() for header in table.find_all("th")]
    rows = []
    for row in table.find_all("tr")[1:]:  # Skip the header row
        cells = row.find_all(["td", "th"])
        row_data = [cell.text.strip() for cell in cells]
        rows.append(row_data)

    return pd.DataFrame(rows, columns=headers).drop(columns=["Map"])


url = "https://en.wikipedia.org/wiki/List_of_Formula_One_circuits"
if html_content := fetch_wiki_circuits(url):
    wiki_circuits = parse_html_to_dataframe(html_content)
    wiki_circuits.to_csv("../data/wiki_circuits.csv", index=False)
else:
    print("Reading from the local file")
    wiki_circuits = pd.read_csv("../data/wiki_circuits.csv")

print("Wikipedia Circuits")
wiki_circuits.head()

Wikipedia Circuits


,Circuit,Type,Direction,Location,Country,Last length used,Turns,Grands Prix,Season(s),Grands Prix held
0,Adelaide Street Circuit,Street circuit,Clockwise,Adelaide,Australia,3.780 km (2.349 mi),16,Australian Grand Prix,1985–1995,11
1,Ain-Diab Circuit,Road circuit,Clockwise,Casablanca,Morocco,7.618 km (4.734 mi),18,Moroccan Grand Prix,1958,1
2,Aintree Motor Racing Circuit,Road circuit,Clockwise,Aintree,United Kingdom,4.828 km (3.000 mi),12,British Grand Prix,"1955, 1957, 1959, 1961–1962",5
3,Albert Park Circuit *,Street circuit,Clockwise,Melbourne,Australia,5.278 km (3.280 mi),16,Australian Grand Prix,"1996–2019, 2022–2024",27
4,Algarve International Circuit,Race circuit,Clockwise,Portimão,Portugal,4.653 km (2.891 mi),15,Portuguese Grand Prix,2020–2021,2


### Pull data from Ergast API (pre-loaded from Kaggle)

In [5]:
ergast_circuits = pd.read_csv("../data/raw/circuits.csv")
print("Ergast API Circuits")
ergast_circuits.head()

Ergast API Circuits


,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park


## Circuit Name Similarity Metrics

Here are some possible options:
- **Levenshtein distance**: Measures the number of single-character edits (insertions, deletions, substitutions) required to change one word into the other. Useful for small variations in spelling. Try this neat visual calculator: [Levenshtein Distance Calculator](https://phiresky.github.io/levenshtein-demo/).
  - Example: “apple pie” and “apple pi” would have a small Levenshtein distance because only one character is different.

- **Damerau-Levenshtein Distance**: Similar to Levenshtein Distance but also considers transpositions of two adjacent characters.
  - Example: “apple pie” and “appel pie” would have a small Damerau-Levenshtein distance due to the transposition of ‘p’ and ‘l’.

- **Jaccard Similarity**: Measures the similarity between two sets by dividing the size of their intersection by the size of their union. It’s useful for identifying common words or terms.
  - Example: “apple pie” and “pie apple” would have a high Jaccard similarity because they contain the same words, despite the order.

$$ Jaccard(U,V) = \frac{|U \cap V|}{|U \cup V|} $$

- **Cosine similarity**: Measures the cosine of the angle between two vectors in a multi-dimensional space. For short texts, this can be implemented using term frequency (TF) vectors.
  - Example: Converts “apple pie” and “pie apple” into vectors and calculates the cosine of the angle between them, which will be close to 1 if they are similar.

$$ Cosine(x,y) = \frac{x \cdot y}{|x||y|} $$

- **TF-IDF with Cosine Similarity**: Term Frequency-Inverse Document Frequency (TF-IDF) considers the importance of a word in a document relative to the entire corpus. Using TF-IDF vectors with cosine similarity helps in identifying similar texts based on the importance of terms.
  - Example: “apple pie recipe” and “recipe for apple pie” would have high similarity as TF-IDF captures the significance of words in the context.

- **Word Embeddings (e.g., Word2Vec, GloVe)**: Word embeddings map words into continuous vector spaces where semantically similar words are close to each other. Average the embeddings of words in the short texts and then use cosine similarity.
  - Example: “apple pie” and “pie with apples” would have similar vector representations in an embedding space.


For short texts with slight differences, a combination of TF-IDF with Cosine Similarity and Jaccard Similarity is often effective. Let's try that.

### Preprocess

In [7]:
def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    tokens = nltk.word_tokenize(text)
    tokens = [
        word for word in tokens if word not in nltk.corpus.stopwords.words("english")
    ]
    lemmatizer = nltk.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)

wiki_circuits["modified_name"] = wiki_circuits["Circuit"].apply(lambda x: preprocess(x))
ergast_circuits["modified_name"] = ergast_circuits["name"].apply(
    lambda x: preprocess(x)
)

### Calculate similarity

In [8]:
# Combine all unique preprocessed names for TF-IDF fitting
all_names = pd.concat(
    [ergast_circuits["modified_name"], wiki_circuits["modified_name"]]
).unique()

# Calculate TF-IDF Cosine Similarity
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(all_names)

# Split the TF-IDF matrix back to individual dataframes
tfidf_ergast = tfidf_matrix[: len(ergast_circuits)]
tfidf_wiki = tfidf_matrix[len(ergast_circuits) :]

# Compute cosine similarity
cosine_similarities = cosine_similarity(tfidf_ergast, tfidf_wiki)

# Find the best matches for fuzzy join
ergast_circuits["best_cos_match"] = ergast_circuits.index.map(
    lambda i: wiki_circuits["modified_name"].iloc[cosine_similarities[i].argmax()]
)
ergast_circuits["cos_similarity_score"] = ergast_circuits.index.map(
    lambda i: cosine_similarities[i].max()
)

ergast_circuits[["name", "best_cos_match", "cos_similarity_score"]]

,name,best_cos_match,cos_similarity_score
0,Albert Park Grand Prix Circuit,aintree motor racing circuit,0.699437
1,Sepang International Circuit,albert park circuit,0.325843
2,Bahrain International Circuit,albert park circuit,0.325843
3,Circuit de Barcelona-Catalunya,autodromo nazionale di monza,0.374864
4,Istanbul Park,charade circuit,0.752447
...,...,...,...
72,Autódromo Internacional do Algarve,albert park circuit,0.516203
73,Autodromo Internazionale del Mugello,autódromo estoril,0.400886
74,Jeddah Corniche Circuit,autódromo oscar juan gálvez,0.081891
75,Losail International Circuit,albert park circuit,0.325843


In [9]:
def jaccard_similarity(str1, str2):
    set1, set2 = set(str1.split()), set(str2.split())
    return len(set1 & set2) / len(set1 | set2)


# Calculate Jaccard Similarity for each pair of rows
import itertools
n_ergast = len(ergast_circuits)
n_wiki = len(wiki_circuits)
jaccard_similarities = np.zeros((n_ergast, n_wiki))

for i, j in itertools.product(range(n_ergast), range(n_wiki)):
    jaccard_similarities[i, j] = jaccard_similarity(
        ergast_circuits["modified_name"].iloc[i],
        wiki_circuits["modified_name"].iloc[j],
    )

# Find the best matches for fuzzy join
ergast_circuits["best_jac_match"] = ergast_circuits.index.map(
    lambda i: wiki_circuits["modified_name"].iloc[jaccard_similarities[i].argmax()]
)
ergast_circuits["jac_similarity_score"] = ergast_circuits.index.map(
    lambda i: jaccard_similarities[i].max()
)

ergast_circuits[["name", "best_cos_match", "cos_similarity_score", "best_jac_match", "jac_similarity_score"]]

,name,best_cos_match,cos_similarity_score,best_jac_match,jac_similarity_score
0,Albert Park Grand Prix Circuit,aintree motor racing circuit,0.699437,albert park circuit,0.600000
1,Sepang International Circuit,albert park circuit,0.325843,sepang international circuit,1.000000
2,Bahrain International Circuit,albert park circuit,0.325843,bahrain international circuit,1.000000
3,Circuit de Barcelona-Catalunya,autodromo nazionale di monza,0.374864,circuit de barcelonacatalunya,1.000000
4,Istanbul Park,charade circuit,0.752447,intercity istanbul park,0.666667
...,...,...,...,...,...
72,Autódromo Internacional do Algarve,albert park circuit,0.516203,autódromo internacional rio de janeiro,0.333333
73,Autodromo Internazionale del Mugello,autódromo estoril,0.400886,autodromo internazionale del mugello,1.000000
74,Jeddah Corniche Circuit,autódromo oscar juan gálvez,0.081891,jeddah corniche circuit,1.000000
75,Losail International Circuit,albert park circuit,0.325843,algarve international circuit,0.500000


In [14]:
def levenshtein_similarity(str1, str2):
    return 1 - (Levenshtein.distance(str1, str2) / max(len(str1), len(str2)))


# Calculate Levenshtein Similarity for each pair of rows
import itertools
n_ergast = len(ergast_circuits)
n_wiki = len(wiki_circuits)
levenshtein_similarities = np.zeros((n_ergast, n_wiki))

for i, j in itertools.product(range(n_ergast), range(n_wiki)):
    levenshtein_similarities[i, j] = levenshtein_similarity(
        ergast_circuits["modified_name"].iloc[i],
        wiki_circuits["modified_name"].iloc[j],
    )

# Find the best matches for fuzzy join
ergast_circuits["best_lev_match"] = ergast_circuits.index.map(
    lambda i: wiki_circuits["modified_name"].iloc[levenshtein_similarities[i].argmax()]
)
ergast_circuits["lev_distance_score"] = ergast_circuits.index.map(
    lambda i: levenshtein_similarities[i].max()
)

ergast_circuits[
    [
        "name",
        "best_cos_match",
        "cos_similarity_score",
        "best_jac_match",
        "jac_similarity_score",
        "best_lev_match",
        "lev_distance_score",
    ]
]

,name,best_cos_match,cos_similarity_score,best_jac_match,jac_similarity_score,best_lev_match,lev_distance_score
0,Albert Park Grand Prix Circuit,aintree motor racing circuit,0.699437,albert park circuit,0.600000,caesar palace grand prix circuit,0.718750
1,Sepang International Circuit,albert park circuit,0.325843,sepang international circuit,1.000000,sepang international circuit,1.000000
2,Bahrain International Circuit,albert park circuit,0.325843,bahrain international circuit,1.000000,bahrain international circuit,1.000000
3,Circuit de Barcelona-Catalunya,autodromo nazionale di monza,0.374864,circuit de barcelonacatalunya,1.000000,circuit de barcelonacatalunya,1.000000
4,Istanbul Park,charade circuit,0.752447,intercity istanbul park,0.666667,intercity istanbul park,0.565217
...,...,...,...,...,...,...,...
72,Autódromo Internacional do Algarve,albert park circuit,0.516203,autódromo internacional rio de janeiro,0.333333,autodromo internazionale del mugello,0.666667
73,Autodromo Internazionale del Mugello,autódromo estoril,0.400886,autodromo internazionale del mugello,1.000000,autodromo internazionale del mugello,1.000000
74,Jeddah Corniche Circuit,autódromo oscar juan gálvez,0.081891,jeddah corniche circuit,1.000000,jeddah corniche circuit,1.000000
75,Losail International Circuit,albert park circuit,0.325843,algarve international circuit,0.500000,lusail international circuit,0.964286


We can see that Jaccard similarity performs the best. This makes sense given that there is a lot of overlap in the names of the circuits as they use common words and roots of words (e.g., international, circuit, park ). This demonstrates that matching terms (words) between the texts are more critical than the frequency or order of words. Also, because terms are not repeated in the names of circuits, it is intuitive that term frequency would not add valuable information. 

### Fuzzy Join

In [18]:
# Perform the fuzzy join based on the best matches
fuzzy_joined_df = ergast_circuits.merge(
    wiki_circuits,
    left_on="best_jac_match",
    right_on="modified_name",
    suffixes=("_ergast", "_wiki"),
)

fuzzy_joined_df.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url,modified_name_ergast,...,Type,Direction,Location,Country,Last length used,Turns,Grands Prix,Season(s),Grands Prix held,modified_name_wiki
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,albert park grand prix circuit,...,Street circuit,Clockwise,Melbourne,Australia,5.278 km (3.280 mi),16,Australian Grand Prix,"1996–2019, 2022–2024",27,albert park circuit
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...,sepang international circuit,...,Race circuit,Clockwise,Sepang,Malaysia,5.543 km (3.444 mi),15,Malaysian Grand Prix,1999–2017,19,sepang international circuit
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...,bahrain international circuit,...,Race circuit,Clockwise,Sakhir,Bahrain,5.412 km (3.363 mi),15,"Bahrain Grand Prix,Sakhir Grand Prix","2004–2010, 2012–2024[a]",21,bahrain international circuit
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...,circuit de barcelonacatalunya,...,Race circuit,Clockwise,Montmeló,Spain,4.657 km (2.894 mi),14,Spanish Grand Prix,1991–2024,34,circuit de barcelonacatalunya
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park,istanbul park,...,Race circuit,Anti-clockwise,Istanbul,Turkey,5.338 km (3.317 mi),14,Turkish Grand Prix,"2005–2011, 2020–2021",9,intercity istanbul park


### Missing data imputation